In [1]:
import pandas as pd
import openpyxl
import sys
sys.path.append("/Users/ipeksahbazoglu/VSCode/ProjectWeave/Modules")

from knowledgebase import KB
from knowledgeembedding import KGE
import pickle

/Users/ipeksahbazoglu/opt/anaconda3/envs/velapartners/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load Success Test Data

Load Investor + Failed/Successful Company Data

In [11]:
df_i = pd.read_excel("investor-df.xlsx")
df_f = pd.read_excel("failed_comp.xlsx")
df_s = pd.read_excel("successful_comp.xlsx")

Set-up Dataframes, add founder_name to comp data etc.

In [13]:
#df_comp = df_s.merge(s_test[['org_uuid', 'founder_name']], on = 'org_uuid')
#df_comp = df_comp[['org_name','founder_name', 'status', 'founded_on','category_list', 'country_code', 'city',
       #'investment_type', 'investors_names',
       #'investor_number', 'investment_size']]
df_comp = df_s[['org_name','founder_name','category_list',
       'investment_type', 'investors_names',
       'investor_number', 'investment_size']]

df_founder = s_test.merge(df_s[['org_uuid']], on = 'org_uuid')
df_founder = df_founder[['founder_name', 'founder_school',
       'founder_degree', 'founder_exp', 'founder_roles', 'founder_conn']]

KeyError: "['founder_name'] not in index"

In [379]:
df_test = df_s[['org_uuid', 'investors_uuids']].merge(s_test[['org_uuid']], on = 'org_uuid')
investors = {}
for ele in df_test['investors_uuids']:
    investors[ele] = []
for i, row in df_i.iterrows():
    if row['investor_uuid'] not in investors:
        df_i.drop(i, axis=0, inplace=True)
df_i.reset_index(drop = True, inplace = True)
df_i.drop(columns = ['investor_uuid', 'investment_count', 'investment_count_normalised'], inplace = True)
    

Create Knowledge Base Class

In [380]:
class KB():
    def __init__(self):
        self.relations = []
        self.entities = {}
    
    #since it is a dictionary we don't need to check for duplicate entries
    #this is useful for building the graph/searching/querying
    def add_entity(self, e):
        self.entities[e] = []
        
    #we need to check for duplicate relations before adding them to the array
    #seperate function that checks duplicates is written
    def are_relations_equal(self, r1, r2):
        #checks if all attributes of the relation is equal
        #all() true if all items are true
        return all(self.fuzzy_equal(r1[attr], r2[attr]) for attr in ["head", "tail"])
    def exists_relation(self, r1):
        #iterates through all the relations in our current relations to see 
        #if the input relation is in there
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)
    #we also need to check if the relation contains a company name, or is a triplet
    #relating to the company, in which case the triplet is related to the company
    #via the description link
    
    def fuzzy_equal(self, str1, str2):
        ratio1 = fuzz.partial_ratio(str1.lower(), str2.lower())
        ratio2 = fuzz.partial_ratio(str1.lower(), str2.lower())
        return (ratio1 > 95) or (ratio2 > 95)

        
    def add_relation(self, r):
        if not self.exists_relation(r):
            if not self.fuzzy_equal(r['head'], r['tail']):
                self.relations.append(r)
                 
                entities = [r["head"], r["tail"]]

                for ent in entities:
                    self.add_entity(ent)

                if 'meta' in r:
                    if self.fuzzy_equal(r["head"], r['meta']['comp']) != self.fuzzy_equal(r["tail"], r['meta']['comp']):
                        if self.fuzzy_equal(r["head"], r['meta']['comp']):
                            r["head"] = r['meta']['comp']
                        else:
                            r2 = {
                            'head': r['meta']['comp'],
                            'type': "description",
                            'tail': r["head"], }
                            r2['meta'] = {
                                "comp": r['meta']['comp']}
                            self.add_relation(r2)

                        if self.fuzzy_equal(r["tail"], r['meta']['comp']):
                            r["tail"] = r['meta']['comp']    
            else:
                return
    #join two knowledge bases
    def merge_with_kb(self, kb2):
        for r in kb2.relations:
            self.add_relation(r)

    def print(self):
        print("Relations:")
        for r in self.relations:
            print(f"  {r}")

Function to create entity and relation links for company/founder/investor data

In [381]:
def extract_comp_data(df, relations = [], v_list = []):
    name = ['investor_name', "org_name"]
    for i in range(df.shape[1]):
        col = df.columns[i]
        if (col not in name):
            for n, ele in enumerate(df[col]):
                    if col in v_list:
                        for item in ele.split(','):
                            relations.append({
                            'head': df['org_name'][n],
                            'type': col,
                            'tail': str(item)
                        })   

                    else: 
                        relations.append({
                        'head': df['org_name'][n],
                        'type': col,
                        'tail': str(ele)
                        })
    kb = KB()
    for relation in relations:
        kb.add_relation(relation)
        
    return kb

In [382]:
def extract_founder_data(df, relations = []):
    name = ['founder_name']
    for i in range(df.shape[1]):
        col = df.columns[i]
        if (col not in name):
            for n, ele in enumerate(df[col]):
                    for item in ele.split(','):
                        relations.append({
                        'head': df['founder_name'][n],
                        'type': col,
                        'tail': str(item) })
    kb = KB()
    for relation in relations:
        kb.add_relation(relation)
        
    return kb

In [383]:
def extract_inv_data(df, relations = [], v_list = []):
    name = ['investor_name', "investor_uuid"]
    for i in range(df.shape[1]):
        col = df.columns[i]
        if (col not in name):
            for n, ele in enumerate(df[col]):
                    if col in v_list:
                        for item in ele.split(','):
                            relations.append({
                            'head': df['investor_name'][n],
                            'type': col,
                            'tail': str(item)
                        })   

                    else: 
                        relations.append({
                        'head': df['investor_name'][n],
                        'type': col,
                        'tail': str(ele)
                        })   
    kb = KB()
    for relation in relations:
        kb.add_relation(relation)
        
    return kb

Get and Merge Knowledge Bases for the Datasets

In [384]:
kb_comp = extract_comp_data(df_comp, v_list = ['category_list', 'investors_names'])
kb_founder = extract_founder_data(df_founder)
kb_inv = extract_inv_data(df_i, v_list = ['investor_types'])

In [385]:
kb_comp.print()

Relations:
  {'head': '10X Banking', 'type': 'founder_name', 'tail': 'Antony Jenkins CBE'}
  {'head': '10X Genomics', 'type': 'founder_name', 'tail': 'Ben Hindson'}
  {'head': '10X Genomics', 'type': 'founder_name', 'tail': 'Serge Saxonov'}
  {'head': '111, Inc.', 'type': 'founder_name', 'tail': 'Junling Liu'}
  {'head': '17zuoye', 'type': 'founder_name', 'tail': 'Dun (Shield) Xiao'}
  {'head': '1inch', 'type': 'founder_name', 'tail': 'Anton Bukov'}
  {'head': '1inch', 'type': 'founder_name', 'tail': 'Sergej Kunz'}
  {'head': '360Learning', 'type': 'founder_name', 'tail': 'Sebastien Mignot'}
  {'head': '360Learning', 'type': 'founder_name', 'tail': 'Nick Hernandez'}
  {'head': '4G Clinical', 'type': 'founder_name', 'tail': 'Ed Tourtellotte'}
  {'head': '4G Clinical', 'type': 'founder_name', 'tail': 'David Kelleher'}
  {'head': '4Paradigm', 'type': 'founder_name', 'tail': 'Shiwei Hu'}
  {'head': '4Paradigm', 'type': 'founder_name', 'tail': 'Yuqiang Chen'}
  {'head': '4Paradigm', 'type':

In [386]:
kb_all = KB()
kb_all.merge_with_kb(kb_inv)
kb_all.merge_with_kb(kb_founder)
kb_all.merge_with_kb(kb_comp)

In [312]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

In [314]:
def extract_relations_from_model_output(text):
    relations = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")
    for token in text_replaced.split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    return relations

In [317]:
def from_row_to_kb(df, span_length=128, verbose=False):
    comp = df['org_name']
    text = df['long_description']
    # tokenize whole text
    inputs = tokenizer([text], return_tensors="pt")
    # compute span boundaries
    num_tokens = len(inputs["input_ids"][0])
    if verbose:
        print(f"Input has {num_tokens} tokens")
    num_spans = math.ceil(num_tokens / span_length)
    if verbose and num_tokens > 1:
        print(f"Input has {num_spans} spans")
    overlap = math.ceil((num_spans * span_length - num_tokens) / 
                        max(num_spans - 1, 1))
    spans_boundaries = []
    start = 0
    for i in range(num_spans):
        spans_boundaries.append([start + span_length * i,
                                 start + span_length * (i + 1)])
        start -= overlap
    #if verbose:
        #print(f"Span boundaries are {spans_boundaries}")

    # transform input with spans
    tensor_ids = [inputs["input_ids"][0][boundary[0]:boundary[1]]
                  for boundary in spans_boundaries]
    tensor_masks = [inputs["attention_mask"][0][boundary[0]:boundary[1]]
                    for boundary in spans_boundaries]
    inputs = {
        "input_ids": torch.stack(tensor_ids),
        "attention_mask": torch.stack(tensor_masks)
    }

    # generate relations
    num_return_sequences = 3
    gen_kwargs = {
        "max_length": 256,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": num_return_sequences
    }
    generated_tokens = model.generate(
        **inputs,
        **gen_kwargs,
    )

    # decode relations
    decoded_preds = tokenizer.batch_decode(generated_tokens,
                                           skip_special_tokens=False)

    # create kb
    kb = KB()
    i = 0
    for sentence_pred in decoded_preds:
        current_span_index = i // num_return_sequences
        relations = extract_relations_from_model_output(sentence_pred)
        for relation in relations:
            relation["meta"] = {
                "comp": comp
            }
            kb.add_relation(relation)
        i += 1
    return kb

In [318]:
def from_desc_to_kb(df, verbose=False):
    kb = KB()
    if verbose:
        print(f"{len(df)} links to visit")
    for i in range(len(df)):
        if verbose:
            print(f"Visiting {descriptions['name'][i]}...")
        kb_comp = from_row_to_kb(df.iloc[i], verbose=verbose)
        kb.merge_with_kb(kb_comp)
        if i%10 == 0:
            print('On row {i}')
    return kb

In [319]:
df_desc = df_s[['org_uuid', 'org_name','long_description']].merge(s_test[['org_uuid']], on = 'org_uuid')
kb_s_desc = from_desc_to_kb(df_desc, verbose=False)


On row {i}
On row {i}
On row {i}
On row {i}
On row {i}
On row {i}
On row {i}
On row {i}
On row {i}
On row {i}


In [296]:
kb_all.merge_with_kb(kb_s_desc)

In [388]:
net = Network(directed=True, width="700px", height="700px", bgcolor="#eeeeee", notebook = True)


for e in kb_all.entities:
    net.add_node(e, shape="circle", color="#00ff1e")


# edges
for r in kb_all.relations:
    net.add_edge(r["head"], r["tail"],
                title=r["type"], label=r["type"])

# save network
net.repulsion(
    node_distance=200,
    central_gravity=0.2,
    spring_length=200,
    spring_strength=0.05,
    damping=0.09
)
net.set_edge_smooth('dynamic')
net.show("network.html")



Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [374]:
net = Network(directed=True, width="700px", height="700px", bgcolor="#eeeeee", notebook = True)


for e in kb_comp.entities:
    net.add_node(e, shape="circle", color="#00ff1e")


# edges
for r in kb_comp.relations:
    net.add_edge(r["head"], r["tail"],
                title=r["type"], label=r["type"])

# save network
net.repulsion(
    node_distance=200,
    central_gravity=0.2,
    spring_length=200,
    spring_strength=0.05,
    damping=0.09
)
net.set_edge_smooth('dynamic')
net.show("network.html")
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [366]:
import pickle

with open('kb.pickle', 'wb') as handle:
    pickle.dump(kb_all, handle, protocol=pickle.HIGHEST_PROTOCOL)



1705